In [35]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

import warnings
warnings.filterwarnings("ignore")

In [36]:
X = np.array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
              [1, 1, 2, 1, 3, 0, 5, 10, 1, 2],
              [500, 700, 750, 600, 1450, 800, 1500, 2000, 450, 1000],
              [1, 1, 2, 1, 2, 1, 3, 3, 1, 2]], dtype = np.float64)

y = np.array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1], dtype = np.float64)

задание 1: Измените функцию calc_logloss так, чтобы нули по возможности не попадали в np.log.

In [37]:
def calc_logloss(y, y_pred):
    exceptions = [-1,0,1]
    err = - np.mean(y * np.log(y_pred) + (1.0 - y) * np.log(1.0 - y_pred))
    for i in exceptions:
        if i in y_pred - y:
            y_pred = np.where([y_pred == 0]  , 0.99999, y_pred)
            err = - np.mean(y * np.log(y_pred) + (1.0 - y) * np.log(1.0 - y_pred))
            return err
    return err

In [38]:
# Пример применения
y1 = np.array([1, 0])
y_pred1 = np.array([0.02, 0.7])
calc_logloss(y1, y_pred1)

2.557997904877041

In [39]:
def sigmoid(z):
    res = 1 / (1 + np.exp(-z))
    return res

In [40]:
X_st = X.copy()

задание 2: Подберите аргументы функции eval_model для логистической регрессии таким образом, чтобы log loss был минимальным.

In [ ]:
# немного не понял задачи - log loss выдает нам вектор весов, как его минимизировать
# минимизация log_loss подразумевает минимизацию ошибки при оптимальном кол-ве итераций?

In [41]:
def eval_model(X, y, iterations, alpha=1e-4):
    np.random.seed(42)
    W = np.random.randn(X.shape[0])
    n = X.shape[1]
    for i in range(1, iterations+1):
        z = np.dot(W, X)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
        W -= alpha * (1/n * np.dot((y_pred - y), X.T))
#     if i % (iterations / 10) == 0:
#         print(i, W, err)
    return err

In [43]:
grid = [(0.1, 1000), (1e-2, 1000), (1e-4, 1000), (1e-5, 1000), (1e-6, 1000)]

for learning_rate, n_iterations in grid:
    err = eval_model(X_st, y, iterations=n_iterations, alpha=learning_rate)
    print(err)

5.756467732512389
76.25045847673358
5.042792352603376
0.8478082963339114
nan


Оптимальное кол-во итераций = 1000 при скорости обучения 1e-5

задание 3: Создайте функцию calc_pred_proba, возвращающую предсказанную вероятность класса 1 (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred_proba).

In [59]:
def eval_model(X, y, iterations, alpha=1e-4):   
    
    np.random.seed(42)
    W = np.random.randn(X.shape[0])
    n = X.shape[1]
    weights_history, error_history = [W], []
    
    for i in range(1, iterations+1):
        z = np.dot(W, X)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
        W -= alpha * (1/n * np.dot((y_pred - y), X.T))
        
        error_history.append(err)
        weights_history.append(W)
#     if i % (iterations / 10) == 0:
#         print(i, W, err)
    return W

def calc_pred_proba(W: np.array, X: np.array):
    """
    
    Input:
    
    W -> вектор весов, полученный в результате обучения модели лог. регрессии
    X -> таблица объект-признак
    
    Output:
    y_pred_proba -> предсказанная вероятность калсса 1
    
    """
    y_pred = np.dot(W, X)
    y_pred_proba = sigmoid(y_pred)
    return y_pred_proba

calc_pred_proba(X_st.T, eval_model(X_st, y, iterations=1000, alpha=1e-5), )

array([0.45091443, 0.26368886, 0.53691502, 0.3516191 , 0.05229711,
       0.21391627, 0.13439504, 0.00957292, 0.50262493, 0.29122261])

задание 4: Создайте функцию calc_pred, возвращающую предсказанный класс (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred).

In [69]:
def calc_pred(W: np.array, X: np.array):
    """    
    Input:
    
    W -> вектор весов, полученный в результате обучения модели лог. регрессии
    X -> таблица объект-признак
    
    Output:
    y_pred -> предсказанный класс: 1 класс присваивается при вероятности >= 0.5; 0 при вероятности < 0.5
    
    """
    threshold = 0.5
    y_pred = np.dot(W, X)
    y_pred_proba = sigmoid(y_pred)
    result = np.where(y_pred_proba >= threshold, 1, 0)
    
    return result

calc_pred(X_st.T, eval_model(X_st, y, iterations=1000, alpha=1e-5))

array([0, 0, 1, 0, 0, 0, 0, 0, 1, 0])

задание 5: Посчитайте Accuracy, матрицу ошибок, точность и полноту, а также F1 score.

In [97]:
def confusion_matrix(y_pred: np.array, y_true: np.array):
    y_pred = calc_pred(X_st.T, eval_model(X_st, y, iterations=1000, alpha=1e-5))
    y_true = y
    n_pred = y_true.shape[0]

    TP = 0
    TN = 0
    FP = 0
    FN = 0
    
    for i in range(n_pred):
        if y_true[i] == 1:            
            if y_pred[i] == 1:
                TP+=1
            else: FN+=1
        elif y_true[i] == 0:
            if y_pred[i] == 0:
                TN +=1
            else: FP+=1
        
    return TP, TN, FP, FN

y_pred = calc_pred(X_st.T, eval_model(X_st, y, iterations=1000, alpha=1e-5))
confusion_matrix(y_pred, y)

(2, 5, 0, 3)

In [104]:
TP, TN, FP, FN = confusion_matrix(y_pred, y)
accuracy = (TP + TN)/(TP + TN + FP + FN)
precision = TP / (TP + FP)
recall = TP / (TP + FN)
F1_score = (2 * precision * recall) / (precision + recall)

print(f'accuracy = {accuracy:.2f}')
print(f'precision = {precision:.2f}')
print(f'recall = {recall:.2f}')
print(f'F1_score = {F1_score:.2f}')

accuracy = 0.70
precision = 1.00
recall = 0.40
F1_score = 0.57


задание 6: Могла ли модель переобучиться? Почему?

В наших данных целевая перемення распределена равномерна, так что моджель вроде не должна переобучаться
Но для полноты высказывания нужно больше объектов и проверить модель на обучающей и тестовой выборках, чтобы потом использовать на валидационной